In [333]:
import requests
from bs4 import BeautifulSoup as bs
import json

def course_info(link, serial_number, course_code):
    info = []
    info.append(serial_number)
    info.append(course_code)
    response = requests.request("GET", url = link)
    soup = bs(response.text, 'html.parser')
    all_trs = soup.find_all('table')
    trs = all_trs[0].find_all('tr')

    index = [1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
    for i in range(len(index)):
        if index[i] == 5:
            try:
                tds = trs[index[i]].find_all('td')
                for td in tds:
                    name_list = list(td.stripped_strings)
                    name_list.append('')
                    s = name_list[0]
                    for j in range(len(name_list) - 2):
                        s = s + ' ' + name_list[j + 1]
                    info.append(s)
            except:
                info.append('無資料')
        else:    
            try:
                tds = trs[index[i]].find_all('td')
                for td in tds:
                    s = td.text.replace('\n', '').replace('\t', '')
                    info.append(s)
            except:
                info.append('無資料')
    info.append(link)
    return info

def course_list(link):
    tmp = []
    response = requests.request("GET", url = link)
    soup = bs(response.text, 'html.parser')
    all_courses = soup.find_all('tbody')
    for courses in all_courses:
        course = courses.find_all('tr')
        for cours in course:
            tds = cours.find_all('td')
            serial_number = tds[0].text[0:5]
            course_code = tds[0].text[5:tds[0].size]
            onclick = tds[-1].a.get("onclick")
            left = onclick.find("'")
            right = onclick.find("'", left+1)
            a = course_info('https://cis.ncu.edu.tw' + onclick[left+1:right], serial_number, course_code)
            tmp.append(a)
    #a = course_info('https://cis.ncu.edu.tw' + onclick[left+1:right])
    return tmp
    #print(len(course))
    
def course_page(link):
    tmp = []
    response = requests.request("GET", url = link)
    soup = bs(response.text, 'html.parser')
    div = soup.find_all('div', class_ = 'pagelinks')
    if (div != []):
        pages = div[0].find_all('a')
        left = link.find("?")  
        for i in range(len(pages)):
            a = course_list(link[0:left+1] + 'd-49489-p=' + str(i+1) + '&' + link[left+1:-1] + '0')
            for data in a:
                tmp.append(data)
    else:
        a = course_list(link)
        for data in a:
            tmp.append(data)

    return tmp


response = requests.request("GET", url = 'https://cis.ncu.edu.tw/Course/main/query/byUnion')
soup = bs(response.text, 'html.parser')
li = soup.find_all('li')

a = []
count = 0
for link in li:
    count = count + 1
    print(count)
    datas = course_page('https://cis.ncu.edu.tw' + link.a.get("href"))
    for data in datas:
        dictionary = {
            'Serial Number': data[0],
            'Course Code': data[1],
            'Semester': data[2],
            'Department': data[3],
            'Instructor': data[4],
            'Course Name (Chinese)': data[5],
            'Course Name (English)': data[6],
            'Educational System': data[7],
            'Credit': data[8],
            'Teaching Goal': data[9],
            'Teaching Content': data[10],
            'Textbooks/References': data[11],
            'Self-compiled Textbook/References Proportion': data[12],
            'Way of Instruction': data[13],
            'Grading': data[14],
            'Office Hour': data[15],
            'Teaching Weeks': data[16],
            'Flexible learning week description': data[17],
            'Course Domain': data[18],
            'Syllabus Link': data[19]
        }
        a.append(dictionary)
with open('output.json', 'w', encoding='utf8') as outfile:
    json.dump(a, outfile, indent=4, ensure_ascii=False)




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
